In [2]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import os
import cv2

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [5]:
# Placeholder variable for the input images
x = tf.placeholder(tf.float32, shape=[None, 28*28], name='X')
# Reshape it into [num_images, img_height, img_width, num_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Placeholder variable for the true labels associated with the images
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


In [6]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, name):
    
    with tf.variable_scope(name) as scope:
        # Shape of the filter-weights for the convolution
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        # Create new weights (filters) with the given shape
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

        # Create new biases, one for each filter
        biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))

        # TensorFlow operation for convolution
        layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')

        # Add the biases to the results of the convolution.
        layer += biases
        
        return layer, weights

In [7]:
def new_pool_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.max_pool(value=input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        return layer

In [8]:
def new_relu_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.relu(input)
        
        return layer

In [9]:
def new_fc_layer(input, num_inputs, num_outputs, name):
    
    with tf.variable_scope(name) as scope:

        # Create new weights and biases.
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05))
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
        
        # Multiply the input and weights, and then add the bias-values.
        layer = tf.matmul(input, weights) + biases
        
        return layer

In [20]:
# Convolutional Layer 1
layer_conv1, weights_conv1 = new_conv_layer(input=x_image, num_input_channels=1, filter_size=5, num_filters=6, name ="conv1")

# Pooling Layer 1
layer_pool1 = new_pool_layer(layer_conv1, name="pool1")

# RelU layer 1
layer_relu1 = new_relu_layer(layer_pool1, name="relu1")

# Convolutional Layer 2
layer_conv2, weights_conv2 = new_conv_layer(input=layer_relu1, num_input_channels=6, filter_size=5, num_filters=16, name= "conv2")

# Pooling Layer 2
layer_pool2 = new_pool_layer(layer_conv2, name="pool2")

# RelU layer 2
layer_relu2 = new_relu_layer(layer_pool2, name="relu2")

# Flatten Layer
num_features = layer_relu2.get_shape()[1:4].num_elements()
layer_flat = tf.reshape(layer_relu2, [-1, num_features])

# Fully-Connected Layer 1
layer_fc1 = new_fc_layer(layer_flat, num_inputs=num_features, num_outputs=128, name="fc1")

# RelU layer 3
layer_relu3 = new_relu_layer(layer_fc1, name="relu3")

# Fully-Connected Layer 2
layer_fc2 = new_fc_layer(input=layer_relu3, num_inputs=128, num_outputs=10, name="fc2")

In [ ]:
# Use Softmax function to normalize the output
with tf.variable_scope("Softmax"):
    y_pred = tf.nn.softmax(layer_fc2)
    y_pred_cls = tf.argmax(y_pred, dimension=1)

In [ ]:
# Use Cross entropy cost function
with tf.name_scope("cross_ent"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
    cost = tf.reduce_mean(cross_entropy)

In [ ]:
# Use Adam Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [ ]:

# Accuracy
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# Trainable variables
vart_list = tf.trainable_variables()

In [ ]:
# Initialize the FileWriter
writer = tf.summary.FileWriter("Training_FileWriter/")
writer1 = tf.summary.FileWriter("Validation_FileWriter/")

In [ ]:
# Add the cost and accuracy to summary
tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', accuracy)

# Add the trainable variables to summary 
for var in vart_list:
    tf.summary.histogram(var.name, var)

# Merge all summaries together
merged_summary = tf.summary.merge_all()

INFO:tensorflow:Summary name conv1/Variable:0 is illegal; using conv1/Variable_0 instead.
INFO:tensorflow:Summary name conv1/Variable_1:0 is illegal; using conv1/Variable_1_0 instead.
INFO:tensorflow:Summary name conv2/Variable:0 is illegal; using conv2/Variable_0 instead.
INFO:tensorflow:Summary name conv2/Variable_1:0 is illegal; using conv2/Variable_1_0 instead.
INFO:tensorflow:Summary name fc1/Variable:0 is illegal; using fc1/Variable_0 instead.
INFO:tensorflow:Summary name fc1/Variable_1:0 is illegal; using fc1/Variable_1_0 instead.
INFO:tensorflow:Summary name fc2/Variable:0 is illegal; using fc2/Variable_0 instead.
INFO:tensorflow:Summary name fc2/Variable_1:0 is illegal; using fc2/Variable_1_0 instead.
INFO:tensorflow:Summary name conv1_1/Variable:0 is illegal; using conv1_1/Variable_0 instead.
INFO:tensorflow:Summary name conv1_1/Variable_1:0 is illegal; using conv1_1/Variable_1_0 instead.
INFO:tensorflow:Summary name conv2_1/Variable:0 is illegal; using conv2_1/Variable_0 ins

In [ ]:
num_epochs = 3
batch_size = 100

In [ ]:
with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    batch_count = 0
    # Loop over number of epochs
    for epoch in range(num_epochs):
        
        start_time = time.time()
        train_accuracy = 0
        num_batch = int(len(data.train.labels)/batch_size)
        for batch in range(0, num_batch):
            
            # Get a batch of images and labels
            x_batch, y_true_batch = data.train.next_batch(batch_size)
            
            # Put the batch into a dict with the proper names for placeholder variables
            feed_dict_train = {x: x_batch, y_true: y_true_batch}
            
            # Run the optimizer using this batch of training data.
            sess.run(optimizer, feed_dict=feed_dict_train)
            
            # Calculate the accuracy on the batch of training data
            #train_accuracy += sess.run(accuracy, feed_dict=feed_dict_train)
            train_accuracy += sess.run(accuracy, feed_dict=feed_dict_train)
            temp_train_accuracy = train_accuracy / (batch+1)
            batch_count += 1
            if((batch_count % 50) == 0):
                print("Batch Size: "+str(epoch+1)+" Steps "+str(batch_count)+",Training Accuracy:\t{0:.3f}".format(temp_train_accuracy))
            
            # Generate summary with the current batch of data and write to file
            summ = sess.run(merged_summary, feed_dict=feed_dict_train)
            writer.add_summary(summ, epoch*int(len(data.train.labels)/batch_size) + batch)
            
          
        train_accuracy /= int(len(data.train.labels)/batch_size)
        
        # Generate summary and validate the model on the entire validation set
#         summ, vali_accuracy = sess.run([merged_summary, accuracy], feed_dict={x:data.validation.images, y_true:data.validation.labels})
#         writer1.add_summary(summ, epoch)
        summ, testing_accuracy = sess.run([merged_summary, accuracy], feed_dict={x:data.test.images, y_true:data.test.labels})
        writer1.add_summary(summ, epoch)
        

        end_time = time.time()
        
        # Save checkpoint of the model
        checkpoint_name = os.path.join("Checkpoints/", 'model_epoch'+str(epoch+1)+'.ckpt')
        save_path = tf.train.Saver().save(sess, checkpoint_name)  
        
        #print("Steps "+str(epoch+1)+",Training Accuracy:\t{}".format(train_accuracy))
        #print(str(epoch)+" "+str(num_epochs))
        if(epoch == num_epochs-1):
            print("Testing Accuracy:\t{}".format(testing_accuracy))
        
        #print("Epoch "+str(epoch+1)+" completed : Time usage "+str(int(end_time-start_time))+" seconds")
        #print("\tAccuracy:")
        #print ("\t- Training Accuracy:\t{}".format(train_accuracy))
        #print ("\t- Validation Accuracy:\t{}".format(vali_accuracy))

Batch Size: 1 Steps 50,Training Accuracy:	0.218
Batch Size: 1 Steps 100,Training Accuracy:	0.330
Batch Size: 1 Steps 150,Training Accuracy:	0.416
Batch Size: 1 Steps 200,Training Accuracy:	0.480
Batch Size: 1 Steps 250,Training Accuracy:	0.532
Batch Size: 1 Steps 300,Training Accuracy:	0.576
Batch Size: 1 Steps 350,Training Accuracy:	0.612
Batch Size: 1 Steps 400,Training Accuracy:	0.641
Batch Size: 1 Steps 450,Training Accuracy:	0.665
Batch Size: 1 Steps 500,Training Accuracy:	0.684
Batch Size: 1 Steps 550,Training Accuracy:	0.700


In [ ]:

# Restore variables weight and calculate accuracy on the batch of validation data 
with tf.Session() as sess:
    
    # Restore the pretrained weights
    tf.train.Saver().restore(sess, "Checkpoints/model_epoch100.ckpt")
    
    # Feed Dictionary
    feed_dict={x:data.validation.images[78:90], y_true:data.validation.labels[78:90]}
    
    # Make the Predictions on the batch of validaton data
    prediction_labels = sess.run(y_pred_cls, feed_dict=feed_dict)

In [ ]:

# Visualize classification using OpenCV
for i, image in enumerate(feed_dict[x]):
    image = np.array(image).reshape(28,28)
    height, width = image.shape[:2]
    image  = cv2.resize(image, (width*10, height*10), interpolation = cv2.INTER_CUBIC)
    
    cv2.putText(image, "True class: "+ str(np.argmax(feed_dict[y_true], axis=1)[i]), (20,30) , cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    cv2.putText(image, "Predict class: "+str(prediction_labels[i]),                  (140,30), cv2.FONT_HERSHEY_SIMPLEX, .5, 1)
    
    cv2.imshow("Images",image)
    cv2.waitKey(3000)
cv2.destroyAllWindows()